In [ ]:
from sqlalchemy import create_engine, Column, Integer, Text, ForeignKey
from sqlalchemy.orm import sessionmaker
# Data injection
engine = create_engine('postgresql://jliu3230@localhost/enshance')
Session = sessionmaker(bind=engine)
session = Session()

# Preprocessing


In [ ]:
results = session.execute('''SELECT * from labelling_titledescriptionsubjects order by id''')
# Preprocessing

results = results.fetchall()
subjects = [result[4] for result in results]
doc_ids = [result[0] for result in results]

In [ ]:
subjects = list(map(lambda x: x.strip('[').strip(']').strip('u\'').strip('\''), subjects))

In [ ]:
import re
subjects_list = list(map(lambda x: re.findall('u\'(.*?)\'', x), subjects))

In [ ]:
ex14001 = subjects[14000]

In [ ]:
ex14001 = clean_doc(ex14001)

In [ ]:
vectorizer2 = CountVectorizer(min_df=1)
vectorizer2.fit_transform([ex14001])

In [ ]:
ex_vector = np.asarray([np.asarray(model[x]) for x in vectorizer2.get_feature_names()])

In [ ]:
ex_vector.shape

In [ ]:
len(ex_vector)

In [ ]:
with open("exvector_noscience", 'wb') as f:
    np.save(f, np.delete(ex_vector, (1), 0))

In [ ]:
doc_ids

In [ ]:
# get taxonomy terms
engine2 = create_engine('postgresql://jliu3230@localhost/sharetaxonomy')
Session2 = sessionmaker(bind=engine2)
session2 = Session2()
results2 = session2.execute('''SELECT term from terms''')
terms = [result for result in results2]

In [ ]:
# Word2Vec
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


In [ ]:
example = subjects_list[10]


In [ ]:
sentences = [subject.split(' ') for subject in example]

In [ ]:
sentences

In [ ]:
model = gensim.models.Word2Vec.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

In [ ]:
terms[0]

In [ ]:
# Keep training
total = 0
missing = []
for tv in terms:
    for t in tv[0].split(" "):
        total += 1
        try: 
            model[t]
        except:
            missing.append(tv)
            break
print(total)
print(missing)

Total words: 20140; missing words: 3003

In [ ]:
model.similarity('mathematics', 'biological')

In [ ]:
from numpy.linalg import norm
norm(model['computer'] - model['physical'])+norm(model['science'] - model['science'])

- Remove stop words from terms
- Remove punctuations
- Split subjects by ", " and " - "


In [ ]:
# Remove stop words from taxonomy terms

# get stop words
import requests
import bs4

html = requests.get("http://www.ranks.nl/stopwords").content
soup = bs4.BeautifulSoup(html)

In [ ]:
soup

In [ ]:
str(soup.find_all('td')[0:2])

In [ ]:
stopwords = str(soup.find_all('td')[0])[17:-5].split("<br/>") + str(soup.find_all('td')[1])[17:-5].split("<br/>")

In [ ]:
# get the collections of terms for each root
# Computer science
collections = {}
roots = (991, 2092, 2911, 2948, 3208, 5930, 7398, 7626, 8600, 8912, 9268)
for root in roots:
    tree = session2.execute('select * from taxonomytree where array_length(path, 1) < 10 and path[1] = {};'.format(root)).fetchall()
    child_terms = [row[3] for row in tree]
    child_ids = [row[2] for row in tree]
    collections[root] = [(tree[0][0], tree[0][1])] + [x for x in zip(child_ids, child_terms)]


In [ ]:
collections[2092]

In [ ]:
# join all terms for each collection
collection_docs = {}
for key in collections.keys():
    collection_docs[key] = " ".join([termtuple[1] for termtuple in collections[key]])

In [ ]:
collection_docs[2092]

In [ ]:
# build corpus
collection_corpus = list(collection_docs.values())
collection_corpus = list(map(clean_doc, collection_corpus))

In [ ]:
collection_corpus[8]

In [ ]:
# analyze docs using scikit learn
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=1)
analyze = vectorizer.build_analyzer()
analyze(collection_corpus[1])

In [ ]:
X = vectorizer.fit_transform(collection_corpus)

In [ ]:
vectorizer.get_feature_names()

In [ ]:
import numpy as np

In [ ]:
num_of_words = np.sum(X.toarray(), axis=1)

In [ ]:
num_of_words

In [ ]:
collection_weights.shape

In [ ]:
collection_weights = X.toarray() / num_of_words[:, None]

In [ ]:
# Make sure weights add up to 1
np.sum(collection_weights, axis=1)

In [ ]:
collection_vectors = [model[x] for x in vectorizer.get_feature_names()]

In [ ]:
collection_vectors[0]

In [ ]:
# serialize
import pickle

In [ ]:
with open("collection_vectors.pickle", 'wb') as f:
    np.save(f, collection_vectors)
with open("collection_weights.pickle", 'wb') as f:
    np.save(f, collection_weights)

In [ ]:
# Now deal with subjects from the data
subjects_list

In [ ]:
# First just try a few example
subjects

In [ ]:
len(collection_vectors)

In [ ]:
collection_docs.values()

In [ ]:
# see if the word science is dominant

vectorizer.get_feature_names()[1588]

In [ ]:
collection_weights[:,1588]

In [ ]:
with open("collection_vectors_noscience", 'wb') as f:
    np.save(f, np.delete(collection_vectors, (1588), 0))
with open("collection_weights_noscience", 'wb') as f:
    np.save(f, np.delete(collection_weights, 1588, 1))

In [ ]:
np.delete(collection_vectors, (1588),0)

In [ ]:
collection_vectors

In [ ]:
doc_dist(ex_vector, np.asarray(collection_vectors)[np.where(collection_weights[0]>0)])


In [ ]:
doc_dist(ex_vector, np.asarray(collection_vectors)[np.where(collection_weights[8]>0)])


In [ ]:
doc_dist(ex_vector, np.asarray(collection_vectors)[np.where(collection_weights[9]>0)])

In [ ]:
for i in range(11):
    print(doc_dist(np.delete(ex_vector,(1),0), np.asarray(collection_vectors)[np.where(collection_weights[i]>0)]))

In [ ]:
np.repeat(ex_vector, len(collection_vectors), 0)

In [ ]:
word_dist(ex_vector, np.asarray(collection_vectors)[np.where(collection_weights[0]>0)])

In [ ]:
list(collection_weights[0]>0)

In [ ]:
np.asarray(collection_vectors)[np.where(collection_weights[0]>0)]

In [ ]:
collection_weights.shape

In [ ]:
### try title and description

titles = [result[1] for result in results]
descriptions = [result[2] for result in results]

In [ ]:
td14001 = clean_doc(clean_doc(titles[14000] + descriptions[14000]))
td14001

In [ ]:
vectorizer3 = CountVectorizer(min_df=1, stop_words=stopwords)
vectorizer3.fit_transform([td14001])

In [ ]:
td_vector = np.asarray([np.asarray(model[x]) for x in vectorizer3.get_feature_names()])

In [ ]:
for i in range(11):
    print(doc_dist(td_vector, np.asarray(collection_vectors)[np.where(collection_weights[i]>0)]))

In [ ]:
title_sample = titles[14002:14021]
description_sample = descriptions[14002:14021]
td_sample = list(map(lambda x: " ".join(x), zip(title_sample, description_sample)))

In [ ]:
td_sample = list(map(clean_doc, td_sample))

In [ ]:
td_sample[1]

In [ ]:
vectorizer3 = CountVectorizer(min_df=1, stop_words=stopwords)
X3 = vectorizer3.fit_transform(td_sample)
min_dist = []
for i in range(19):
    td_sample_vector = np.asarray([np.asarray(model[x]) for x in np.asarray(vectorizer3.get_feature_names())[np.where(X3.toarray()[i,:]>0)]])
    distances = [doc_dist(td_sample_vector, np.asarray(collection_vectors)[np.where(collection_weights[j]>0)]) for j in range(11)]
    val, idx = min((val, idx) for (idx, val) in enumerate(distances))
    min_dist += (val,idx)

In [ ]:
print(min_dist)

In [ ]:
np.asarray(vectorizer3.get_feature_names())[np.where(X3[1,:]>0)]

In [ ]:
X3.toarray()[1,:]

In [ ]:
# Try basic word count method
sample_vocab = np.asarray(vectorizer3.get_feature_names())
collection_vocab = np.asarray(vectorizer.get_feature_names())
sample_X = X3.toarray()
collection_X = X.toarray()


In [ ]:
for sample_dist in sample_X:
    sample_words = list(sample_vocab[np.where(sample_dist>0)])
    print(sample_words)
    count = []
    min_dist = []
    for collection_dist in collection_X:
        collection_words = list(collection_vocab[np.where(collection_dist>0)])
        # print(len([x for x in sample_words if x in collection_words]))
        print(len([x for x in sample_words if x in collection_words]))
        print([x for x in sample_words if x in collection_words])
    #min_dist += (min((val, idx) for (idx, val) in enumerate(count)))

In [ ]:
print(min_dist)

In [ ]:
# Prepare data: subjects, title and description, title description subjects

**Test functions here**

In [ ]:
list((map(clean_doc, td_sample)))

In [ ]:
label_docs(td_sample, vectorizer)

In [ ]:
label_docs(td_sample, vectorizer, method='wmd')

In [ ]:
label_docs(td_sample, vectorizer, method='count_weight')

In [ ]:
# Prepare documents
# read labels
truelabelscsv = np.genfromtxt ('truelabels.csv', delimiter=",").astype(int)


In [ ]:
all_labels = list(enumerate(collection_docs.keys()))

In [ ]:
# subjects only, title_description
# sample ids
sample_ids = truelabelscsv[:,0]
truelabels = truelabelscsv[:,1]

In [ ]:
# get all the rows from result
sample = np.asarray(results)[sample_ids-1,:]

In [ ]:
# subjects
sample_subjects = sample[:,4]
# title
sample_titles = sample[:,1]
# description
sample_descriptions = sample[:,2]
# title_descriptions
sample_title_descriptions = sample[:,3]

In [ ]:
# all
sample_all = list(map(lambda x: " ".join(x), zip(sample_descriptions, sample_subjects)))


In [ ]:
# labelling for each set of documents using all methods
methods = ['count', 'count_weight', 'wmd']


In [ ]:
model_s = np.array([label_docs(sample_subjects,vectorizer,method=x) for x in methods])


In [ ]:
model_t = np.array([label_docs(sample_titles,vectorizer,method=x) for x in methods])
model_d = np.array([label_docs(sample_descriptions,vectorizer,method=x) for x in methods])
model_td = np.array([label_docs(sample_title_descriptions,vectorizer,method=x) for x in methods])

In [ ]:
model_all = np.array([label_docs(sample_all,vectorizer,method=x) for x in methods])

In [ ]:
# Pull predicted labels
pred_s = pull_labels(model_s)

In [ ]:
pred_t = pull_labels(model_t)
pred_d = pull_labels(model_d)
pred_td = pull_labels(model_td)

In [ ]:
pred_all = pull_labels(model_all)

In [ ]:
# Calculate accuracy
accuracy_s = [get_accuracy(pred_s[i], truelabels) for i in range(3)]

In [ ]:
accuracy_t = [get_accuracy(pred_t[i], truelabels) for i in range(3)]
accuracy_d = [get_accuracy(pred_d[i], truelabels) for i in range(3)]
accuracy_td = [get_accuracy(pred_td[i], truelabels) for i in range(3)]

In [ ]:
accuracy_all = [get_accuracy(pred_all[i], truelabels) for i in range(3)]

In [ ]:
print(accuracy_s)
print(accuracy_td)
print(accuracy_all)

In [ ]:
pred_all[2] == truelabels

In [ ]:
np.asarray([accuracy_s[0],accuracy_td[0],accuracy_all[0]])[np.newaxis].transpose()

In [ ]:
np.array(pred_s[0],pred_td[0],pred_all[0])

In [ ]:
# Add true labels to the results
table = np.column_stack((sample, truelabels))

In [ ]:
# Put labels together. Only use subjects, title_descriptions
# order by method, then by type of documents
for i in range(3):
    table = np.column_stack((table,pred_s[i]))
    table = np.column_stack((table,pred_td[i]))
    table = np.column_stack((table,pred_all[i]))

In [ ]:
table

In [ ]:
import csv
with open('sample_table.csv', 'w') as f:
    fieldnames = ['id','title','description','title_description','subjects','manual_label', 'by_subjects_count', 'by_title_desc_count', 'by_all_count', 'by_subjects_count_weight', 'by_title_desc_count_weight', 'by_all_count_weight', 'by_subjects_wmd', 'by_title_desc_wmd', 'by_all_wmd']
    writer = csv.writer(f)
    writer.writerow(fieldnames)
    writer.writerows(table)

In [ ]:
# Plot accuracy

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
np.random.seed(1974)

# Generate Data
num = 20
x, y = np.random.random((2, num))
labels = np.random.choice(['a', 'b', 'c'], num)
df = pd.DataFrame(dict(x=x, y=y, label=labels))

groups = df.groupby('label')

# Plot
fig, ax = plt.subplots()
ax.margins(0.05) # Optional, just adds 5% padding to the autoscaling
for name, group in groups:
    ax.plot(group.x, group.y, marker='o', linestyle='', ms=12, label=name)
ax.legend()

plt.show()

**Define all functions here**

In [ ]:
# clean docs
def clean_doc(doc, stop_words):
    letters_only = re.sub("[^a-zA-Z]", " ", doc)
    letters_only = re.sub("  ", " ", letters_only)
    letters_list = letters_only.lower().split(" ")
    no_stopwords = [w for w in letters_list if w not in stopwords and w in model]
    return " ".join(no_stopwords)


In [ ]:
import itertools

def word_dist(v1, v2):
    return norm(v1-v2)
def doc_dist(d1, d2):
    dist = 0
    for u in d1:
        dist += min(list(map(word_dist, np.repeat(u,len(d2),0), d2)))
    return dist

In [ ]:
# write a function that does all the above jobs given any doc list
def label_docs(docs, tax_vectorizer, method='count'):
    """given a method (count, count_weight, wmd), label the docs using information in the taxonomy.
    docs: a list of documents to be labelled.
    tax_vectorizer: sk-learn vectorizer for the taxonomy
    method: can be count, count_weight, wmd.
    return the index of the label in collection_corpus and corresponding metrics"""
    docs = list((map(clean_doc, docs)))
    docs_vectorizer = CountVectorizer(min_df=1, stop_words=stopwords)
    docs_X = docs_vectorizer.fit_transform(docs).toarray()
    docs_vocab = np.asarray(docs_vectorizer.get_feature_names())
    tax_vocab = np.asarray(tax_vectorizer.get_feature_names())
    tax_X = X.toarray()
    if method == 'count':
        max_count = []
        for doc_distribution in docs_X:
            doc_words = list(docs_vocab[np.where(doc_distribution>0)])
            count = []
            for collection_distribution in tax_X:
                collection_words = list(tax_vocab[np.where(collection_distribution>0)])
                # print(len([x for x in sample_words if x in collection_words]))
                count.append(len([x for x in doc_words if x in collection_words]))
            max_count.append((max((val, idx) for (idx, val) in enumerate(count))))
        return max_count
    if method == 'count_weight':
        max_count = []
        for doc_distribution in docs_X:
            doc_words = list(docs_vocab[np.where(doc_distribution>0)])
            count = []
            for i, collection_distribution in enumerate(tax_X):
                collection_words = list(tax_vocab[np.where(collection_distribution>0)])
                # print(len([x for x in sample_words if x in collection_words]))
                cocurrence = [x for x in doc_words if x in collection_words]
                count.append(sum([collection_weights[i, idx] for idx, val in enumerate(tax_vocab) if val in cocurrence]))
            max_count.append((max((val, idx) for (idx, val) in enumerate(count))))
        return max_count
    if method == 'wmd':
        min_dist = []
        for i in range(len(docs)):
            doc_vector = np.asarray([np.asarray(model[x]) for x in np.asarray(docs_vectorizer.get_feature_names())[np.where(docs_X[i,:]>0)]])
            distances = [doc_dist(doc_vector, np.asarray(collection_vectors)[np.where(collection_weights[j]>0)]) for j in range(11)]
            val, idx = min((val, idx) for (idx, val) in enumerate(distances))
            min_dist.append((val,idx))
        return min_dist
    

In [ ]:
# get labels from the results
def pull_labels(ob):
    return np.asarray(list(map(lambda o: np.asarray([all_labels[int(x[1])][1] for x in o]), ob)))

In [ ]:
def get_accuracy(pred, test):
    """Percentage of correct labels"""
    return np.sum(pred==test)/len(test)

In [ ]:
[x for x in collection_corpus]
labels = [()]

In [ ]:
all_labels

In [ ]:
np.sum(pred_s[2]==truelabels)/len(truelabels)